In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import StandardScaler, PowerTransformer, QuantileTransformer, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
os.chdir('/content/drive/My Drive/Bootcamp ML2020/')
os.listdir('.')
dataset = pd.read_csv('bottle.csv')
dataset.head(100)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (47,73) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,BtlNum,RecInd,T_prec,T_qual,S_prec,S_qual,P_qual,O_qual,SThtaq,O2Satq,ChlorA,Chlqua,Phaeop,Phaqua,PO4uM,PO4q,SiO3uM,SiO3qu,NO2uM,NO2q,NO3uM,NO3q,NH3uM,NH3q,C14As1,C14A1p,C14A1q,C14As2,C14A2p,C14A2q,DarkAs,DarkAp,DarkAq,MeanAs,MeanAp,MeanAq,IncTim,LightP,R_Depth,R_TEMP,R_POTEMP,R_SALINITY,R_SIGMA,R_SVA,R_DYNHT,R_O2,R_O2Sat,R_SIO3,R_PO4,R_NO3,R_NO2,R_NH4,R_CHLA,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1,DIC Quality Comment
0,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.50,33.440,NaN,25.649,NaN,NaN,NaN,3,1.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,0.0,10.50,10.50,33.440,25.64,233.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.440,NaN,25.656,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,8.0,10.46,10.46,33.440,25.65,232.5,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,NaN,25.654,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,10.0,10.46,10.46,33.437,25.65,232.8,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.420,NaN,25.643,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,19.0,10.45,10.45,33.420,25.64,234.1,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,NaN,25.643,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,20.0,10.45,10.45,33.421,25.64,234.0,0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4,96,050.0 095.0,19-4903CR-HY-061-1042-05000950-0019A-3,19,10.14,32.660,NaN,25.103,NaN,NaN,NaN,3,2.0,NaN,2.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,19.0,10.14,10.14,32.660,25.10,285.4,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,4,97,050.0 095.0,19-4903CR-HY-061-1042-05000950-0020A-7,20,10.14,32.657,NaN,25.100,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,20.0,10.14,10.14,32.657,25.10,285.7,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,4,98,050.0 095.0,19-4903CR-HY-061-1042-05000950-0030A-7,30,10.07,32.641,NaN,25.100,NaN,NaN,NaN,7,2.0,NaN,3.0,NaN,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,30.0,10.07,10.07,32.641,25.10,285.9,0.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,4,99,050.0 095.0,19-4903CR-HY-061-1042-05000950-0040A-3,40,9.97,NaN,NaN,NaN,NaN,NaN,NaN,3,2.0,NaN,NaN,9.0,9.0,9.0,9.0,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,40.0,9.97,9.97,NaN,NaN,NaN,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
dataset.describe()

,Cst_Cnt,Btl_Cnt,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,BtlNum,RecInd,T_prec,T_qual,S_prec,S_qual,P_qual,O_qual,SThtaq,O2Satq,ChlorA,Chlqua,Phaeop,Phaqua,PO4uM,PO4q,SiO3uM,SiO3qu,NO2uM,NO2q,NO3uM,NO3q,NH3uM,NH3q,C14As1,C14A1p,C14A1q,C14As2,C14A2p,C14A2q,DarkAs,DarkAp,DarkAq,MeanAs,MeanAp,MeanAq,LightP,R_Depth,R_TEMP,R_POTEMP,R_SALINITY,R_SIGMA,R_SVA,R_DYNHT,R_O2,R_O2Sat,R_SIO3,R_PO4,R_NO3,R_NO2,R_NH4,R_CHLA,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1
count,864863.000000,864863.000000,864863.000000,853900.000000,817509.000000,696201.000000,812174.000000,661274.000000,661268.000000,118667.000000,864863.000000,853900.000000,23127.000000,817509.000000,74914.000000,673755.000000,184676.000000,65823.00000,217797.000000,225272.000000,639166.000000,225271.000000,639170.000000,413317.000000,451786.000000,354091.000000,510866.000000,337576.000000,529474.000000,337403.000000,529933.000000,64962.000000,808299.000000,14432.000000,12760.000000,848605.000000,14414.000000,12742.000000,848623.000000,22649.000000,20457.000000,840440.000000,22650.000000,20457.000000,840439.000000,18651.000000,864863.000000,853900.000000,818816.000000,817509.000000,812007.000000,812092.000000,818206.000000,696201.000000,666448.000000,354099.000000,413325.000000,337411.000000,337584.000000,64982.000000,225276.000000,225275.000000,864863.000000,122006.000000,1999.000000,224.000000,2084.000000,234.000000,10.000000,84.000000
mean,17138.790958,432432.000000,226.831951,10.799677,33.840350,3.392468,25.819394,57.103779,148.808694,10.497426,4.700273,2.017148,7.471829,2.716787,7.945057,8.999996,8.755615,8.46701,8.793487,0.450185,8.999848,0.198553,8.999844,1.564747,8.999320,26.606587,8.999687,0.042316,8.995458,17.302247,8.993122,0.084877,8.947668,9.757704,1.266928,8.999992,9.762613,1.267776,8.999992,0.155295,1.993450,8.999992,8.393530,1.256587,8.999992,18.356509,226.832495,10.799697,10.842295,33.840295,25.811010,220.938918,0.431763,3.392498,57.192940,26.606862,1.564736,17.302188,0.042336,0.084851,0.450225,0.198599,228.395694,162.071521,2153.239714,2168.148330,2256.055845,2278.858803,7.948570,7.910983
std,10240.949817,249664.587267,316.050259,4.243825,0.461843,2.073256,1.167787,37.094137,90.187533,6.189688,1.877428,0.148498,1.483097,0.450559,1.415355,0.003655,0.810973,1.11778,0.892111,1.208570,0.012318,0.376543,0.012507,1.035966,0.026895,27.648190,0.018768,0.104196,0.077426,14.581958,0.131516,0.273061,0.508844,23.983492,0.442372,0.002872,25.064529,0.442817,0.002872,0.224953,0.080671,0.002886,21.516374,0.436760,0.002886,29.852839,316.050007,4.243829,4.236435,0.462978,1.124645,93.483703,0.382336,2.073271,37.060537,27.648403,1.035973,14.581998,0.104201,0.273022,1.208566,0.376539,319.456731,85.722796,112.995202,154.852332,34.844435,58.496495,0.021216,0.077666
min,1.000000,1.000000,0.000000,1.440000,28.431000,-0.010000,20.934000,-0.100000,-0.434900,0.000000,3.000000,1.000000,6.000000,2.000000,6.000000,6.000000,6.000000,6.00000,2.000000,-0.001000,8.000000,-3.890000,8.000000,0.000000,4.000000,0.000000,4.000000,0.000000,4.000000,-0.400000,4.000000,0.000000,4.000000,-0.240000,1.000000,8.000000,-0.200000,1.000000,8.000000,-0.010000,1.000000,8.000000,-0.220000,1.000000,8.000000,0.000000,0.000000,1.440000,0.000000,4.570000,20.934000,0.400000,0.000000,-0.010000,-0.100000,0.000000,0.000000,-0.400000,0.000000,0.000000,-0.010000,-3.890000,0.000000,0.000000,1948.850000,1969.440000,2181.570000,2198.150000,7.923100,7.618300
25%,8269.000000,216216.500000,46.000000,7.680000,33.488000,1.360000,24.965000,21.100000,60.915470,5.000000,3.000000,2.000000,6.000000,2.000000,6.000000,9.000000,9.000000,9.00000,9.000000,0.050000,9.000000,0.050000,9.000000,0.470000,9.000000,3.110000,9.000000,0.000000,9.000000,0.600000,9.000000,0.000000,9.000000,0.930000,1.000000,9.000000,0.930000,1.000000,9.000000,0.060000,2.000000,9.000000,0.980000,1.000000,9.000000,0.280000,46.000000,7.680000,7.740000,33.488000,24.960000,143.700000,0.130000,1.360000,21.200000,3.100000,0.470000,0.600000,0.000000,0.000000,

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864863 entries, 0 to 864862
Data columns (total 74 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Cst_Cnt              864863 non-null  int64  
 1   Btl_Cnt              864863 non-null  int64  
 2   Sta_ID               864863 non-null  object 
 3   Depth_ID             864863 non-null  object 
 4   Depthm               864863 non-null  int64  
 5   T_degC               853900 non-null  float64
 6   Salnty               817509 non-null  float64
 7   O2ml_L               696201 non-null  float64
 8   STheta               812174 non-null  float64
 9   O2Sat                661274 non-null  float64
 10  Oxy_µmol/Kg          661268 non-null  float64
 11  BtlNum               118667 non-null  float64
 12  RecInd               864863 non-null  int64  
 13  T_prec               853900 non-null  float64
 14  T_qual               23127 non-null   float64
 15  S_prec           

In [9]:
dataset.describe(include=['O']) 

,Sta_ID,Depth_ID,IncTim,DIC Quality Comment
count,864863,864863,14437,55
unique,2634,864850,199,37
top,090.0 045.0,20-16045S-MX-110-1421-06670900-0003A-3,12/30/1899 05:45:00,CTD Bottle Misfire; DIC sample compromised. C...
freq,10043,2,325,9


In [10]:
nulls = dataset.isnull().sum().sort_values(ascending = False)
prcet = round(nulls/len(dataset)*100,2)

dataset_null = pd.DataFrame(columns =  ['Attr','Total','Percent'])
dataset_null.Attr  = nulls.index
dataset_null.Total = nulls.values
dataset_null.Percent = prcet.values
print(dataset_null.head(20))

                   Attr   Total  Percent
0                   pH2  864853   100.00
1   DIC Quality Comment  864808    99.99
2                   pH1  864779    99.99
3                  DIC2  864639    99.97
4                   TA2  864629    99.97
5                  DIC1  862864    99.77
6                   TA1  862779    99.76
7                C14A2p  852121    98.53
8                C14A1p  852103    98.52
9                C14As2  850449    98.33
10               C14As1  850431    98.33
11               IncTim  850426    98.33
12               LightP  846212    97.84
13               DarkAp  844406    97.63
14               MeanAp  844406    97.63
15               DarkAs  842214    97.38
16               MeanAs  842213    97.38
17               T_qual  841736    97.33
18                NH3uM  799901    92.49
19                R_NH4  799881    92.49


In [11]:
for i in dataset_null.Attr[dataset_null['Percent'] > 30]:
    dataset = dataset.drop([i], axis=1)
    #print(dataset.shape,i)

In [12]:
for i in dataset.columns:
    if dataset[i].isnull().sum() > 0:
        dataset[i].fillna(dataset[i].mode().mean(), inplace=True)

In [13]:
nulls = dataset.isnull().sum().sort_values(ascending = False)
prcet = round(nulls/len(dataset)*100,2)

dataset_null = pd.DataFrame(columns =  ['Attr','Total','Percent'])
dataset_null.Attr  = nulls.index
dataset_null.Total = nulls.values
dataset_null.Percent = prcet.values
print(dataset_null.head())

       Attr  Total  Percent
0    R_PRES      0      0.0
1   R_O2Sat      0      0.0
2   Btl_Cnt      0      0.0
3    Sta_ID      0      0.0
4  Depth_ID      0      0.0


In [14]:
dataset['Year'] = (dataset['Depth_ID'].str.split('-', expand=True)[0] + \
                dataset['Depth_ID'].str.split('-', expand=True)[1]). \
                map(lambda x: str(x)[:4])
dataset['Month'] = (dataset['Depth_ID'].str.split('-', expand=True)[1]). \
                 map(lambda x: str(x)[2:4])
                 
dataset[['Depth_ID','Year','Month']].head(10)

,Depth_ID,Year,Month
0,19-4903CR-HY-060-0930-05400560-0000A-3,1949,03
1,19-4903CR-HY-060-0930-05400560-0008A-3,1949,03
2,19-4903CR-HY-060-0930-05400560-0010A-7,1949,03
3,19-4903CR-HY-060-0930-05400560-0019A-3,1949,03
4,19-4903CR-HY-060-0930-05400560-0020A-7,1949,03
5,19-4903CR-HY-060-0930-05400560-0030A-7,1949,03
6,19-4903CR-HY-060-0930-05400560-0039A-3,1949,03
7,19-4903CR-HY-060-0930-05400560-0050A-7,1949,03
8,19-4903CR-HY-060-0930-05400560-0058A-3,1949,03
9,19-4903CR-HY-060-0930-05400560-0075A-7,1949,03


In [15]:
drop_cols = ['Cst_Cnt', 'Btl_Cnt', 'Sta_ID', 'Depth_ID', 'Depthm','Year','Month']
dataset_norm = dataset.drop(drop_cols, axis=1)  #  data for normalization
dataset_scale = dataset_norm.copy(deep=True)  #  backup data

In [16]:
Y = dataset_norm['T_degC'] #label
x = dataset_norm.drop(['T_degC'], axis=1) #features

**Soal Nomor 1**

In [17]:
def acc_scaler(arr_name, arr_scl):
  # scaler = arr_scl
  X = arr_scl.fit_transform(x) #x=features
  data_x = pd.DataFrame(X, columns=x.columns) #pengganti features di materi

  x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size=0.25, random_state=10)

  lm = LinearRegression()
  lm.fit(x_train, y_train)
  y_pred = lm.predict(x_test)
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  print(arr_name)
  print()
  print('Accuracy of ',arr_name,' on train set : {:.2f}'.format(lm.score(x_train, y_train))) # .score() -> Accuracy
  print('Accuracy of ',arr_name,' on test set  : {:.2f}'.format(lm.score(x_test, y_test))) # .score() -> Accuracy
  print()
  print("MSE (Mean Squared Error)                   :", mse)
  print("MAE (Mean Absolute Error)                  :", mae)
  print("r^2 score                                  :", r2)
  print('RMSE (Root Mean Squared Error)             :', np.sqrt(mean_squared_error(y_test, y_pred)))
  print('====================================================================')

arr_scl=[StandardScaler(),PowerTransformer(method='yeo-johnson', standardize=True),QuantileTransformer(),MinMaxScaler(),MaxAbsScaler(),RobustScaler()]
arr_name=['Standar Scaler','Power Transformer Yeo Johnson','Quantile Transformer','Min Max Scaler','Max Abs Scaler','Robust Scaler']

for a in range(len(arr_scl)):
  acc_scaler(arr_name[a], arr_scl[a])

Standar Scaler

Accuracy of  Standar Scaler  on train set : 1.00
Accuracy of  Standar Scaler  on test set  : 1.00

MSE (Mean Squared Error)                   : 8.674688471347613e-07
MAE (Mean Absolute Error)                  : 0.00011716443433842378
r^2 score                                  : 0.9999999514913407
RMSE (Root Mean Squared Error)             : 0.0009313800766254136


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


Power Transformer Yeo Johnson

Accuracy of  Power Transformer Yeo Johnson  on train set : 1.00
Accuracy of  Power Transformer Yeo Johnson  on test set  : 0.99

MSE (Mean Squared Error)                   : 0.17828402905506238
MAE (Mean Absolute Error)                  : 0.16490761080354957
r^2 score                                  : 0.9900303979163674
RMSE (Root Mean Squared Error)             : 0.4222369347357741
Quantile Transformer

Accuracy of  Quantile Transformer  on train set : 0.97
Accuracy of  Quantile Transformer  on test set  : 0.97

MSE (Mean Squared Error)                   : 0.5471310646813424
MAE (Mean Absolute Error)                  : 0.5010990888319888
r^2 score                                  : 0.9694045561378769
RMSE (Root Mean Squared Error)             : 0.7396830839497024
Min Max Scaler

Accuracy of  Min Max Scaler  on train set : 1.00
Accuracy of  Min Max Scaler  on test set  : 1.00

MSE (Mean Squared Error)                   : 8.674640881617114e-07
MAE (Mean A

**Soal nomor 2**

Random Forest Regressor

In [21]:
def ex_scaler(sclr):
  scaler = sclr
  X = scaler.fit_transform(x) #x=features
  data_x = pd.DataFrame(X, columns=x.columns) #pengganti features di materi

  x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size=0.25, random_state=10)

  arr_r2=[]
  arr_rmse=[]
  arr_estm=[]
  arr_rstatm=[]


  def random_param(est,rstat):
    rf = RandomForestRegressor(n_estimators=est,random_state=rstat)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('\trandom forest n_estimasi = ',est,'dan random_state = ',rstat)
    print("\tr^2 score                      :", r2)
    print('\tRMSE (Root Mean Squared Error) :', rmse)
    print('================================================================')
    arr_r2.append(r2)
    arr_rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    arr_estm.append(est)
    arr_rstatm.append(rstat)

  arr_est=[50] #dikurangi karna lama trainingnya
  arr_rstat=[0]
  for i in arr_est:
    for j in arr_rstat:
      random_param(i,j)

  max_arr2=max(arr_r2)
  # print(max_arr2)

  for i,j in enumerate(arr_r2):
    if max_arr2 == j:
      print('nilai r2 terbaik berada pada model random forest dengan n_estimasi',arr_estm[i],'dan random_state',arr_rstatm[i])
      print('dengan nilai r2 sebesar: ',arr_r2[i])
      print('dan dengan RMSE sebesar: ',arr_rmse[i])

sclr=[StandardScaler(),PowerTransformer(method='yeo-johnson', standardize=True),QuantileTransformer(n_quantiles=10, random_state=0),MinMaxScaler(),MaxAbsScaler(),RobustScaler()]
nm_sclr=['Standar Scaler','Power Transformer Yeo Johnson','Quantile Transformer','Min Max Scaler','Max Abs Scaler','Robust Scaler']

for a in range(len(sclr)):
  print('Untuk Scaler: ',nm_sclr[a])
  print('---------------------------')
  ex_scaler(sclr[a])
  print('*************************************************************************************************************************************************')

Untuk Scaler:  Standar Scaler
---------------------------
	random forest n_estimasi =  50 dan random_state =  0
	r^2 score                      : 0.9999999713755271
	RMSE (Root Mean Squared Error) : 0.0007154611507835643
nilai r2 terbaik berada pada model random forest dengan n_estimasi 50 dan random_state 0
dengan nilai r2 sebesar:  0.9999999713755271
dan dengan RMSE sebesar:  0.0007154611507835643
*************************************************************************************************************************************************
Untuk Scaler:  Power Transformer Yeo Johnson
---------------------------


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


	random forest n_estimasi =  50 dan random_state =  0
	r^2 score                      : 0.9999999727261896
	RMSE (Root Mean Squared Error) : 0.0006983774638122561
nilai r2 terbaik berada pada model random forest dengan n_estimasi 50 dan random_state 0
dengan nilai r2 sebesar:  0.9999999727261896
dan dengan RMSE sebesar:  0.0006983774638122561
*************************************************************************************************************************************************
Untuk Scaler:  Quantile Transformer
---------------------------
	random forest n_estimasi =  50 dan random_state =  0
	r^2 score                      : 0.9999999673788994
	RMSE (Root Mean Squared Error) : 0.0007637770637071927
nilai r2 terbaik berada pada model random forest dengan n_estimasi 50 dan random_state 0
dengan nilai r2 sebesar:  0.9999999673788994
dan dengan RMSE sebesar:  0.0007637770637071927
****************************************************************************************************

**XGBoost**

In [22]:
def ex_scaler(sclr):
  scaler = sclr
  X = scaler.fit_transform(x) #x=features
  data_x = pd.DataFrame(X, columns=x.columns) #pengganti features di materi

  x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size=0.25, random_state=10)

  arr_r2=[]
  arr_rmse=[]
  arr_estm=[]
  arr_lratem=[]


  def random_param(est,lrate):
    xr = xgb.XGBRegressor(n_estimators=est,learning_rate=lrate,objective ='reg:squarederror')
    xr.fit(x_train, y_train)
    y_pred = xr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('\tXGB Regressor n_estimasi = ',est,'dan learning_rate = ',lrate)
    print("\tr^2 score                      :", r2)
    print('\tRMSE (Root Mean Squared Error) :', rmse)
    print('================================================================')
    arr_r2.append(r2)
    arr_rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    arr_estm.append(est)
    arr_lratem.append(lrate)

  arr_est=[50] #sementara saya kurangi dulu estimatornya dan learning ratenya karena memakan waktu yang sangat lama saat running
  arr_lrate=[0.1]
  for i in arr_est:
    for j in arr_lrate:
      random_param(i,j)

  max_arr2=max(arr_r2)
  # print(max_arr2)

  for i,j in enumerate(arr_r2):
    if max_arr2 == j:
      print('nilai r2 terbaik berada pada model XGB Regressor dengan n_estimasi',arr_estm[i],'dan learning_rate',arr_lratem[i])
      print('dengan nilai r2 sebesar: ',arr_r2[i])
      print('dan dengan RMSE sebesar: ',arr_rmse[i])

sclr=[StandardScaler(),PowerTransformer(method='yeo-johnson', standardize=True),QuantileTransformer(n_quantiles=10, random_state=0),MinMaxScaler(),MaxAbsScaler(),RobustScaler()]
nm_sclr=['Standar Scaler','Power Transformer Yeo Johnson','Quantile Transformer','Min Max Scaler','Max Abs Scaler','Robust Scaler']

for a in range(len(sclr)):
  print('Untuk Scaler: ',nm_sclr[a])
  print('---------------------------')
  ex_scaler(sclr[a])
  print('*************************************************************************************************************************************************')

Untuk Scaler:  Standar Scaler
---------------------------
	XGB Regressor n_estimasi =  50 dan learning_rate =  0.1
	r^2 score                      : 0.9996787880935604
	RMSE (Root Mean Squared Error) : 0.07579021258979216
nilai r2 terbaik berada pada model XGB Regressor dengan n_estimasi 50 dan learning_rate 0.1
dengan nilai r2 sebesar:  0.9996787880935604
dan dengan RMSE sebesar:  0.07579021258979216
*************************************************************************************************************************************************
Untuk Scaler:  Power Transformer Yeo Johnson
---------------------------


/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


	XGB Regressor n_estimasi =  50 dan learning_rate =  0.1
	r^2 score                      : 0.9996787880935604
	RMSE (Root Mean Squared Error) : 0.07579021258979216
nilai r2 terbaik berada pada model XGB Regressor dengan n_estimasi 50 dan learning_rate 0.1
dengan nilai r2 sebesar:  0.9996787880935604
dan dengan RMSE sebesar:  0.07579021258979216
*************************************************************************************************************************************************
Untuk Scaler:  Quantile Transformer
---------------------------
	XGB Regressor n_estimasi =  50 dan learning_rate =  0.1
	r^2 score                      : 0.9996787880935604
	RMSE (Root Mean Squared Error) : 0.07579021258979216
nilai r2 terbaik berada pada model XGB Regressor dengan n_estimasi 50 dan learning_rate 0.1
dengan nilai r2 sebesar:  0.9996787880935604
dan dengan RMSE sebesar:  0.07579021258979216
************************************************************************************************

**SVM**

In [ ]:
def ex_scaler(sclr):
  scaler = sclr
  X = scaler.fit_transform(x) #x=features
  data_x = pd.DataFrame(X, columns=x.columns) #pengganti features di materi

  x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size=0.25, random_state=10)

  arr_r2=[]
  arr_rmse=[]
  arr_kernelm=[]


  def random_param(krnl):
    svmr = SVR(kernel=krnl)
    svmr.fit(x_train, y_train)
    y_pred = svmr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('\tSVR dengan kernel = ',krnl)
    print("\tr^2 score                      :", r2)
    print('\tRMSE (Root Mean Squared Error) :', rmse)
    print('================================================================')
    arr_r2.append(r2)
    arr_rmse.append(rmse)
    arr_kernelm.append(krnl)

  arr_kernel=['rbf','poly','linear']
  for i in arr_kernel:
    random_param(i)

  max_arr2=max(arr_r2)
  # print(max_arr2)

  for i,j in enumerate(arr_r2):
    if max_arr2 == j:
      print('nilai r2 terbaik berada pada model SVM dengan kernel',arr_kernelm[i])
      print('dengan nilai r2 sebesar: ',arr_r2[i])
      print('dan dengan RMSE sebesar: ',arr_rmse[i])

sclr=[StandardScaler(),PowerTransformer(method='yeo-johnson', standardize=True),QuantileTransformer(n_quantiles=10, random_state=0),MinMaxScaler(),MaxAbsScaler(),RobustScaler()]
nm_sclr=['Standar Scaler','Power Transformer Yeo Johnson','Quantile Transformer','Min Max Scaler','Max Abs Scaler','Robust Scaler']

for a in range(len(sclr)):
  print('Untuk Scaler: ',nm_sclr[a])
  print('---------------------------')
  ex_scaler(sclr[a])
  print('*************************************************************************************************************************************************')

**KNN**

In [ ]:
def ex_scaler(sclr):
  scaler = sclr
  X = scaler.fit_transform(x) #x=features
  data_x = pd.DataFrame(X, columns=x.columns) #pengganti features di materi

  x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size=0.25, random_state=10)

  arr_r2=[]
  arr_rmse=[]
  arr_kernelm=[]


  def random_param(knnr):
    knn = KNeighborsRegressor(knnr)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('\tKNN dengan nilai k = ',knnr)
    print("\tr^2 score                      :", r2)
    print('\tRMSE (Root Mean Squared Error) :', rmse)
    print('================================================================')
    arr_r2.append(r2)
    arr_rmse.append(rmse)
    arr_kernelm.append(knnr)

  arr_kernel=[1,3,5,7,9] #sementara saya kurangi dulu estimatornya karena memakan waktu yang sangat lama saat running
  for i in arr_kernel:
    random_param(i)

  max_arr2=max(arr_r2)
  # print(max_arr2)

  for i,j in enumerate(arr_r2):
    if max_arr2 == j:
      print('nilai r2 terbaik berada pada model KNN dengan nilai k',arr_kernelm[i])
      print('dengan nilai r2 sebesar: ',arr_r2[i])
      print('dan dengan RMSE sebesar: ',arr_rmse[i])

sclr=[StandardScaler(),PowerTransformer(method='yeo-johnson', standardize=True),QuantileTransformer(n_quantiles=10, random_state=0),MinMaxScaler(),MaxAbsScaler(),RobustScaler()]
nm_sclr=['Standar Scaler','Power Transformer Yeo Johnson','Quantile Transformer','Min Max Scaler','Max Abs Scaler','Robust Scaler']

for a in range(len(sclr)):
  print('Untuk Scaler: ',nm_sclr[a])
  print('---------------------------')
  ex_scaler(sclr[a])
  print('*************************************************************************************************************************************************')